In [1]:
import tensorflow as tf

In [2]:
import keras
from keras import callbacks

Using TensorFlow backend.


In [36]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dropout, Dense, Activation

In [37]:
import numpy as np
import pandas as pd
from keras.utils import normalize

train, test = pd.read_csv('kdd/binary/Training.csv'), pd.read_csv('kdd/binary/Testing.csv')

X,Y = train.iloc[:,1:42], train.iloc[:,0] #data, labels
C,T = test.iloc[:,0], test.iloc[:,1:42]

In [5]:
x_tr, x_te = np.array(X).astype(float), np.array(T).astype(float)

In [6]:
x_tr

array([[0.  , 1.  , 1.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 1.  , 1.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 1.  , 1.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 1.  , 1.  , ..., 0.01, 0.  , 0.  ],
       [0.  , 1.  , 1.  , ..., 0.01, 0.  , 0.  ],
       [0.  , 1.  , 1.  , ..., 0.01, 0.  , 0.  ]])

In [7]:
x_tr = normalize(x_tr) #normalize to zero one scale
x_te = normalize(x_te)

In [9]:
print("X_Tr : ", x_tr,"\nX_Te : ",x_te)

('X_Tr : ', array([[0.00000000e+00, 1.84373540e-03, 1.84373540e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.36287997e-04, 7.36287997e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.37480373e-04, 7.37480373e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 8.04035169e-04, 8.04035169e-04, ...,
        8.04035169e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.92909963e-04, 7.92909963e-04, ...,
        7.92909963e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.81573245e-04, 7.81573245e-04, ...,
        7.81573245e-06, 0.00000000e+00, 0.00000000e+00]]), '\nX_Te : ', array([[ 0.,  2., 24., ...,  0.,  0.,  0.],
       [ 0.,  2., 24., ...,  0.,  0.,  0.],
       [ 0.,  2., 24., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  2., 24., ...,  0.,  0.,  0.],
       [ 0.,  2., 24., ...,  0.,  0.,  0.],
       [ 0.,  2., 24.

In [10]:
y_tr, y_te = np.array(Y), np.array(C)

In [11]:
type(x_tr) #confirm types before building network

numpy.ndarray

In [16]:
x_tr[0].shape #pass this as param for input shape

(41,)

In [35]:
from keras.callbacks import ModelCheckpoint
def build_model(): #build all shallow networks in one function
    
    import time
    dense_layers = [1024,768, 512, 256, 128]
    i=1
    for layer in dense_layers :
        
        model= Sequential()

        model.add(Dense(1024, input_shape=x_tr[0].shape ))
        model.add(Activation('relu'))
        model.add(Dropout(0.01))
        
        if layer!=1024:
            for i in range(1,dense_layers.index(layer)+1):
                model.add(Dense(dense_layers[i]))
                model.add(Activation('relu'))
                model.add(Dropout(0.01))
        
        MODEL_NAME = "{}-hidden_layers-{}-lowest_node-dnn-{}".format(i, layer, int(time.time()))
        
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        
        print ("Model Name : ", MODEL_NAME)
        print model.layers #check model layers before building
        ckpt = ModelCheckpoint("best_acc_weights.hdf5",save_weights_only = True, save_best_only = True, monitor = "val_acc")
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'],callbacks = [ckpt]) #adam is used to optimize the threshold weights
        model.fit(x_tr, y_tr, validation_data= (x_te, y_te), batch_size=128, epochs=10)
        model.save(MODEL_NAME)
        print ("Model {} saved \n").format(MODEL_NAME)
    
build_model()

Model Name :  1-hidden_layers-1024-lowest_node-dnn-1571088673
[<keras.layers.core.Dense object at 0x7feeead07c50>, <keras.layers.core.Activation object at 0x7fef608098d0>, <keras.layers.core.Dropout object at 0x7feeead07e50>, <keras.layers.core.Dense object at 0x7feeead07e10>, <keras.layers.core.Activation object at 0x7fef280c7bd0>]
Epoch 1/10
494020/494020 [==============================] - 17s 34us/step - loss: 0.0267 - acc: 0.9896
Epoch 2/10
494020/494020 [==============================] - 16s 33us/step - loss: 0.0112 - acc: 0.9970
Epoch 3/10
494020/494020 [==============================] - 18s 36us/step - loss: 0.0090 - acc: 0.9973
Epoch 4/10
494020/494020 [==============================] - 17s 34us/step - loss: 0.0080 - acc: 0.9975
Epoch 5/10
494020/494020 [==============================] - 16s 33us/step - loss: 0.0071 - acc: 0.9978
Epoch 6/10
494020/494020 [==============================] - 17s 34us/step - loss: 0.0066 - acc: 0.9980
Epoch 7/10
494020/494020 [=====================

In [38]:
def calc_accuracy(): #calculate all accuracies in one function
    
    import sklearn.metrics
    from sklearn.metrics import (precision_score, recall_score, f1_score, accuracy_score, mean_squared_error, mean_absolute_error, roc_curve, classification_report, auc)
    
    pred_results = [1,2,3,4,5]
    train_data = pd.read_csv('dnnres/expected.txt', header=None)
    
    for item in pred_results:
        
        file_name = 'dnnres/dnn{}predicted.txt'.format(item)
        test_data = pd.read_csv(file_name, header=None)
        
        y_actual, y_predicted = train_data, test_data
        
        accuracy = accuracy_score(y_actual, y_predicted)
        recall = recall_score(y_actual, y_predicted, average='binary')
        precision = precision_score(y_actual, y_predicted, average='binary')
        f1 = f1_score(y_actual, y_predicted, average='binary')
        
        print ("{} layer results : ").format(item)
        print ("Accuracy : %.3f" %accuracy)
        print ("Precision : %.3f" %precision)
        print ("Recall : %.3f" %recall)
        print ("F1 score : %.3f" %f1)
        print ("*******************************\n\n")

In [39]:
calc_accuracy()

1 layer results : 
Accuracy : 0.929
Precision : 0.998
Recall : 0.915
F1 score : 0.954
*******************************


2 layer results : 
Accuracy : 0.929
Precision : 0.998
Recall : 0.914
F1 score : 0.954
*******************************


3 layer results : 
Accuracy : 0.930
Precision : 0.997
Recall : 0.915
F1 score : 0.955
*******************************


4 layer results : 
Accuracy : 0.929
Precision : 0.999
Recall : 0.913
F1 score : 0.954
*******************************


5 layer results : 
Accuracy : 0.927
Precision : 0.998
Recall : 0.911
F1 score : 0.953
*******************************


